In [ ]:
import numpy as np
import pandas as pd
import regex as re
import random as rn
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,average_precision_score, precision_score,precision_recall_curve
from tqdm.notebook import tqdm
from tqdm import trange
import warnings
import pickle
import nltk
import math
import os
import json
import random
import re
import torch
import torch.nn as nn
import spacy 
from spacy import displacy
#import neptune.new as neptune                          
from torch.utils.data import (DataLoader, RandomSampler, WeightedRandomSampler, SequentialSampler, TensorDataset, Dataset)

In [ ]:
!pip install emoji -q
!pip install datasets -q
!pip install rouge_score -q


!pip install transformers -q
!pip install sentencepiece -q
!pip install pandarallel -q

from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel

from transformers import AdamW, get_linear_schedule_with_warmup,PegasusConfig, PegasusTokenizer, PegasusForConditionalGeneration
from transformers import AutoTokenizer, AutoModel, set_seed, TrainingArguments, Trainer, default_data_collator, AdamW, get_linear_schedule_with_warmup, Seq2SeqTrainer, Seq2SeqTrainingArguments


     |████████████████████████████████| 170 kB 27.2 MB/s 
     |████████████████████████████████| 298 kB 26.7 MB/s 
     |████████████████████████████████| 243 kB 41.6 MB/s 
     |████████████████████████████████| 61 kB 487 kB/s 
     |████████████████████████████████| 1.1 MB 35.8 MB/s 
     |████████████████████████████████| 132 kB 65.6 MB/s 
     |████████████████████████████████| 271 kB 46.4 MB/s 
     |████████████████████████████████| 160 kB 53.1 MB/s 
     |████████████████████████████████| 192 kB 35.2 MB/s 
     |████████████████████████████████| 3.4 MB 30.5 MB/s 
     |████████████████████████████████| 596 kB 41.8 MB/s 
     |████████████████████████████████| 3.3 MB 41.2 MB/s 
     |████████████████████████████████| 895 kB 29.7 MB/s 
     |████████████████████████████████| 1.2 MB 34.0 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Mounted at /content/drive


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

ARTICLE_TO_SUMMARIZE = "My friends are cool but they eat too many carbs."
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')

summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=5, early_stopping=True)
print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])


Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


['My friends']


In [ ]:
reviews_path = '/content/drive/MyDrive/gen_datasets/hotel_travel_reviews.json'
# reviews_path = '/content/drive/MyDrive/restaurant_reviews/res_reviews_final.json'
import json
f = open(reviews_path)
# data = json.load(f)
data = json.load(f)
x = data

In [ ]:
# x = {}
# for key_ in data.keys():
#   for a in data[key_]['review_list']:
#     if(a['rating']!=None):
#       x[a['text']] = a['rating']

In [ ]:
sorted_vals = sorted(x.items(), key=lambda item: item[1])

In [ ]:
reviews = []
for j in sorted_vals:
  reviews.append(j[0])

In [ ]:
num_kp = int(len(reviews)/50)
input = []
for i in range(len(reviews)):
  if(i%num_kp==0):
    input.append(reviews[i])
    src_text = ''.join(input)
    inputs = tokenizer([src_text], max_length=1024, return_tensors='pt')
    summary_ids = model.generate(inputs['input_ids'], num_beams=2, max_length=40, early_stopping=True)
    tgt_text = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    print(i,tgt_text)
    input = []

0 ["The food at Pop Century sucks! I can't believe the kind of food they have. It's disgusting and way over priced for the crap it is. You're practically forced to eat"]
71 ['Thought this was a good place to stay  but it was full of mold, cracks and rust in my room. Though it says free breakfast there is NONE  they said it']
142 ['I reserved a car in advanced on line from Budget. When my flight landed, I stood in line for 1 hour and 15 mins waiting. When I finally got to the counter, the']
213 ['Rented a Silverado pick up truck. It smelled like an ashtray and a dirty diaper. The passenger door handle was sticky, garbage under the seats and dirty windshield on the']
284 ["Budget car rental counter at MCO airport has the longest line ever. It's been over 30 minutes and the line hasn't moved. Any other rental counter has a minimal to no"]
355 ['I waited in line for an hour, and at the counter for and hour. They said they didnt have the type of vehicle I had reserved and pre-paid for. I go